In [1]:
import pandas as pd
import numpy as np
import re
import time

# Import SnowballStemmer to remove plural words
from nltk.stem.snowball import SnowballStemmer

# Import translation API from our function dictionary
from dictionary import translate

In [2]:
stemmer = SnowballStemmer("english")

In [3]:
# Prepare all raw data
raw = pd.read_csv('raw_data.csv') # Contains all text crawl from data science website
simple_words = pd.read_csv('simple_words.csv') # Simple words which used to clean raw data

In [4]:
# Prepare dataframe of simple words for filter later
simple_words = simple_words.head(3000) # Only need 3000 most popular words as simple words
simple_words.dropna(inplace = True)
simple_words["word"] = simple_words["word"].apply(lambda x: x.upper())
simple_words["singular"] = simple_words["word"].apply(lambda x: stemmer.stem(x).upper()) # Remove plural from word => become singular. So we can use it to clean raw data
simple_words

,word,count,singular
0,THE,23135851162,THE
1,OF,13151942776,OF
2,AND,12997637966,AND
3,TO,12136980858,TO
4,A,9081174698,A
...,...,...,...
2995,SEXO,25849932,SEXO
2996,AP,25843002,AP
2997,PATTERNS,25809816,PATTERN
2998,BOXES,25787744,BOX


In [5]:
paragraphs = pd.Series(raw["paragraph"]) # All the paragraphs from raw data. This will be used to extract sentence contains data science words.
paragraphs

0       There are many facets to working in Data Scien...
1       Data preparation is particularly important. Da...
2       I then joined PageGroup. There, I worked as an...
3       Now I am a lead big data and machine learning ...
4       JustGiving is still a start-up at heart, so th...
                              ...                        
5395    In the comics, Dr. Stephen Strange, MD, become...
5396    Black boxes are his first choice for everythin...
5397    Vision is an android created by. His role as a...
5398    The real important point, here, is that Vision...
5399    J.A.R.V.I.S. They have decided to test the wat...
Name: paragraph, Length: 5400, dtype: object

In [6]:
# function that filter word with length > 3 and not in simple words
def filter_word(w):
    stemmed_word = stemmer.stem(w).upper()
    
    if len(w) > 3 and w.upper() not in simple_words["word"].values and w.upper() not in simple_words["singular"].values and stemmed_word not in simple_words["singular"].values and stemmed_word not in simple_words["word"].values:
        return True
    return False

In [7]:
# function to clean paragraph
def clean_paragraph(para):
    # loop the string
    arr = re.split('[\W+\d+]', para)
    # transform each word in arr above: filter to get words with length > 3 then uppercase all words
    new_arr = list(map(lambda y: y.upper(), (filter(filter_word, arr))))
    # remove duplicate words
    new_arr = list(dict.fromkeys(new_arr))
    return new_arr

In [8]:
raw["paragraph_arr"] = raw["paragraph"].apply(clean_paragraph)

In [9]:
raw = raw.drop(columns=['paragraph'])
raw

,title,paragraph_arr
0,"How To Work In Data Science, AI, Big Data - KD...","[FACETS, DEPEND, PURSUE, SUCCEED]"
1,"How To Work In Data Science, AI, Big Data - KD...","[SCIENTISTS, TYPICALLY, SHAPED, HAPPIER]"
2,"How To Work In Data Science, AI, Big Data - KD...","[PAGEGROUP, ARCHITECT, TRANSFORMATION, ANALYTICS]"
3,"How To Work In Data Science, AI, Big Data - KD...","[JUSTGIVING, RAISE, ACQUIRED]"
4,"How To Work In Data Science, AI, Big Data - KD...","[JUSTGIVING, TYPICAL, CAPTURE, PIPELINE, INVES..."
...,...,...
5395,Data Avengers… Assemble! - KDnuggets,"[STEPHEN, STRANGE, SORCERER, SUPREME, GUARDIAN..."
5396,Data Avengers… Assemble! - KDnuggets,"[IRIS, DATASET, NEURAL, ENSEMBLE, PREFERS, STA..."
5397,Data Avengers… Assemble! - KDnuggets,"[ANDROID, AVENGER, AUTOMATED, HYBRID, BAYESIAN..."
5398,Data Avengers… Assemble! - KDnuggets,"[SUPPLANTED, COMPLEMENTARY, FLESHY, SCIENTISTS..."


In [10]:
lst_col = 'paragraph_arr'
raw = pd.DataFrame({
    col:np.repeat(raw[col].values, raw[lst_col].str.len())
    for col in raw.columns.difference([lst_col])
}).assign(**{lst_col:np.concatenate(raw[lst_col].values)})[raw.columns.tolist()]
raw

,title,paragraph_arr
0,"How To Work In Data Science, AI, Big Data - KD...",FACETS
1,"How To Work In Data Science, AI, Big Data - KD...",DEPEND
2,"How To Work In Data Science, AI, Big Data - KD...",PURSUE
3,"How To Work In Data Science, AI, Big Data - KD...",SUCCEED
4,"How To Work In Data Science, AI, Big Data - KD...",SCIENTISTS
...,...,...
35386,Data Avengers… Assemble! - KDnuggets,SCIENTISTS
35387,Data Avengers… Assemble! - KDnuggets,UNEMPLOYED
35388,Data Avengers… Assemble! - KDnuggets,WINK
35389,Data Avengers… Assemble! - KDnuggets,COGNITIVE


In [11]:
raw['freq'] = 1

In [12]:
final = raw.groupby('paragraph_arr').agg({
    'freq': 'sum',
    'title': "nunique",
})

In [13]:
final["rank"] = final["freq"] * final["title"]
final = final.sort_values(by=['rank'], ascending=False).reset_index().head(600)

In [14]:
def get_meaning(w):
    meaning, example = translate(w)
    return pd.Series([meaning, example])

In [15]:
final[["meaning", "example"]] = final["paragraph_arr"].apply(get_meaning)

tionaryapi.dev/api/v2/entries/en_US/DEVIATION
https://api.dictionaryapi.dev/api/v2/entries/en_US/RAPID
https://api.dictionaryapi.dev/api/v2/entries/en_US/SATISFACTION
https://api.dictionaryapi.dev/api/v2/entries/en_US/CORRESPONDING
https://api.dictionaryapi.dev/api/v2/entries/en_US/LEARNERS
https://api.dictionaryapi.dev/api/v2/entries/en_US/ENSEMBLE
https://api.dictionaryapi.dev/api/v2/entries/en_US/CHURN
https://api.dictionaryapi.dev/api/v2/entries/en_US/STATISTICIAN
https://api.dictionaryapi.dev/api/v2/entries/en_US/WIDER
https://api.dictionaryapi.dev/api/v2/entries/en_US/AUTOMATE
https://api.dictionaryapi.dev/api/v2/entries/en_US/PARALLEL
https://api.dictionaryapi.dev/api/v2/entries/en_US/ANALYZED
https://api.dictionaryapi.dev/api/v2/entries/en_US/BIASED
https://api.dictionaryapi.dev/api/v2/entries/en_US/ARGUE
https://api.dictionaryapi.dev/api/v2/entries/en_US/EASE
https://api.dictionaryapi.dev/api/v2/entries/en_US/BEGINNERS
https://api.dictionaryapi.dev/api/v2/entries/en_US/WAREHOU

In [16]:
final = final[final["meaning"] != ""]

In [17]:
for word in final['paragraph_arr']:
    for paragraph in paragraphs:
        arr = paragraph.split(".")
        for sentence in arr:
            if word in sentence.upper():
                final.loc[lambda final: final['paragraph_arr'] == word,["sentence"]] = sentence
            break

In [18]:
final = final.reset_index(drop=True)

In [19]:
conditions = [
    final.index <= round(len(final) / 4),
    final.index <= round(2 * len(final) / 4),
    final.index <= round(3 * len(final) / 4),
    final.index <= len(final) - 1
]
choices = ['1', '2', '3', '4']
final['rating'] = np.select(conditions, choices)

In [20]:
final

,paragraph_arr,freq,title,rank,meaning,example,sentence,rating
0,SCIENTISTS,491,271,133061,A person who is studying or has expert knowled...,a research scientist,Don't torture your data scientists by witholdi...,1
1,SCIENTIST,430,235,101050,A person who is studying or has expert knowled...,,More certain am I of this than I am that Marve...,1
2,PYTHON,286,163,46618,A large heavy-bodied nonvenomous snake occurri...,Anacondas are related to boa constrictors and ...,Munge data? Run some analysis? Whip up a class...,1
3,SOLVE,156,118,18408,"Find an answer to, explanation for, or means o...",the policy could solve the town's housing crisis,More certain am I of this than I am that Marve...,1
4,ALGORITHM,140,99,13860,A process or set of rules to be followed in ca...,a basic algorithm for division,More certain am I of this than I am that Marve...,1
...,...,...,...,...,...,...,...,...
543,META,11,11,121,short for meta key,"If you frequently use Alt keys-for example, if...","But then again, in the tech world women are se...",4
544,VARIED,11,11,121,Incorporating a number of different types or e...,a long and varied career,Recent work on k-nearest neighbor (KNN) regres...,4
545,SUBSEQUENT,11,11,121,Coming after something in time; following.,the theory was developed subsequent to the ear...,"Since it is a living blog, I will be writing s...",4
546,INTENSIVE,11,11,121,"An intensive adjective, adverb, or particle; a...","Particles are added, usually as completives an...",Manual labelling is labour-intensive,4


In [21]:
final.to_csv('output.csv', index=False)